# AI Flappy Bird Analysis

## Flappy Bird no-GUI game engine

### Game-specific imports and constants

In [ ]:
import random
import time
import json
import csv
from google.colab import files
import datetime
from zoneinfo import ZoneInfo

GAME_WIDTH = 500
GAME_HEIGHT = 700
MAX_EPISODES = 3

### Classes

#### Bird class

In [ ]:
class Bird:
  def __init__(self, game):
    self.game = game

    self.isFlapping = False

    self.x = self.game.GAME_WIDTH / 2 - 50
    self._y = self.game.GAME_HEIGHT / 2 - 70

    self.width = 50
    self.height = 40

    self.gravity = 600
    self.velocity = 0

  @property
  def y(self):
    return self._y;
  @y.setter
  def y(self, v):
    if (v > self.game.GAME_HEIGHT or v < 0 - self.height):
      self.game.gameIsOver = True
    else:
     self._y = v;

  @property
  def hitbox(self):
    return {
      'x': self.x,
      'y': self.y,
      'width': self.width,
      'height': self.height,
    }

  def update(self, dt):
    self.fall(dt)

  def reset(self):
    self.x = self.game.GAME_WIDTH / 2 - 50
    self._y = self.game.GAME_HEIGHT / 2 - 70

    self.velocity = 0

  def fall(self, dt):
    self.velocity += self.gravity * dt
    self.y += self.velocity * dt

  def flapWings(self):
    self.velocity = -300


#### Pipes class

In [ ]:
class Pipes:
  def __init__(self, game):
    self.game = game

    self.startPoint = 500

    self.gap = 200

    self._x = self.startPoint
    self.topY = random.random() * (self.game.GAME_HEIGHT - self.gap)
    self.botY = self.topY + self.gap

    self.width = 75
    self.topHeight = self.topY
    self.botHeight = self.game.GAME_HEIGHT - self.botY

    self.velocity = 100

    self.passed = False

  @property
  def x(self):
    return self._x
  @x.setter
  def x(self, v):
    if (v + self.width < 0):
      self.game.removePipePair(self)
    else:
      self._x = v

  @property
  def topHitbox(self):
    return {
      'x': self.x,
      'y': 0,
      'width': self.width,
      'height': self.topHeight,
    }
  @property
  def botHitbox(self):
    return {
      'x': self.x,
      'y': self.botY,
      'width': self.width,
      'height': self.botHeight,
    }

  def update(self, dt):
    self.x -= self.velocity * dt

  def reset(self):
    self.x = self.startPoint
    self.topY = random.random() * (self.game.GAME_HEIGHT - self.gap)
    self.botY = self.topY + self.gap

    self.topHeight = self.topY
    self.botHeight = self.game.GAME_HEIGHT - self.botY


#### Game class

In [ ]:
class Game:
  def __init__(self, GAME_WIDTH, GAME_HEIGHT):
    self.accumulator = 0.0

    self.frameStates = []
    self.episodeId = 0
    self.frameId = 0
    self.agentType = 'human'

    self.pipesInterval = 3
    self.flappingInterval = 0.5
    self.deltaTime = 0
    self.lastTime = None
    self.pipesElapsed = 0
    self.flappingElapsed = 0

    self.GAME_WIDTH = GAME_WIDTH
    self.GAME_HEIGHT = GAME_HEIGHT

    self.bird = Bird(self)
    self.pipes = []

    self.gameIsOver = False

    self.points = 0

  def update(self, dt):
    for pipePair in self.pipes:
      if self.areColliding(self.bird.hitbox, pipePair.topHitbox) or \
      self.areColliding(self.bird.hitbox, pipePair.botHitbox):
        self.gameIsOver = True
        return

      if not(pipePair.passed) and self.bird.x > pipePair.x:
        self.points += 1
        pipePair.passed = True

      pipePair.update(dt)
    self.bird.update(dt)

  def gameLoop(self):
    currentTime = time.time()
    if self.lastTime is None:
      self.lastTime = currentTime
    dt = currentTime - self.lastTime
    self.deltaTime = dt
    self.lastTime = currentTime

    FIXED_DT = 0.0167

    self.accumulator += dt

    while self.accumulator >= FIXED_DT:
      if not(self.gameIsOver):
        self.update(FIXED_DT)
      self.accumulator -= FIXED_DT

    if self.gameIsOver:
      if self.episodeId + 1 >= MAX_EPISODES:
        return

      self.gameIsOver = False
      self.reset()
    else:
      self.frameStates.append({
        'episode_id': self.episodeId,
        'frame_id': self.frameId,
        'agent_type': self.agentType,
        'bird_x': self.bird.x,
        'bird_y': self.bird.y,
        'bird_velocity': self.bird.velocity,
        'action': 1 if self.bird.isFlapping else 0,
        'pipe_pair_number': len(self.pipes),
        'pipes': json.dumps([
          {
            "x": pipe_pair.x,
            "top_y": pipe_pair.topY,
            "bot_y": pipe_pair.botY,
            "passed": pipe_pair.passed
          }
          for pipe_pair in self.pipes
        ]) if len(self.pipes) > 0 else 'null',
        'score': self.points,
        'game_is_over': 1 if self.gameIsOver else 0,
      })

      self.pipesElapsed += dt

      if self.pipesElapsed >= self.pipesInterval:
        self.addPipePair()
        self.pipesElapsed = 0

      self.bird.isFlapping = False;
      self.frameId += 1

  def reset(self):
    self.episodeId += 1
    self.frameId = 0

    self.deltaTime = 0
    self.lastTime = None
    self.pipesElapsed = 0
    self.flappingElapsed = 0

    self.bird.reset()

    self.pipes = []

    self.points = 0

  def addPipePair(self):
    pipePair = Pipes(self)
    self.pipes.append(pipePair)
  def removePipePair(self, pipePair):
    if pipePair in self.pipes:
      self.pipes.remove(pipePair)

  def areColliding(self, bird, pipe):
    return bird['x'] < pipe['x'] + pipe['width'] and \
      bird['x'] + bird['width'] > pipe['x'] and \
      bird['y'] < pipe['y'] + pipe['height'] and \
      bird['y'] + bird['height'] > pipe['y'] - 30


### Start and export game

In [ ]:
game = Game(GAME_WIDTH, GAME_HEIGHT)

while not(game.gameIsOver) or game.episodeId + 1 < MAX_EPISODES:
  game.gameLoop()
  time.sleep(0.0167)

now = datetime.datetime.now(ZoneInfo("Europe/Zagreb"))

pad = lambda n: str(n).zfill(2)

current_date = f"{now.year}{pad(now.month)}{pad(now.day)}"
current_time = f"{pad(now.hour)}{pad(now.minute)}{pad(now.second)}"

timestamp = now.strftime("%Y%m%d_%H%M%S")
filename = f"flappy_bird_game_{timestamp}.csv"

headers = list(game.frameStates[0].keys())

with open(filename, 'w', newline="", encoding="utf-8") as csv_file:
  writer = csv.writer(csv_file)
  writer.writerow(headers)
  writer.writerows([[frame[h] for h in headers] for frame in game.frameStates])

files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# NOTES
- rekreirat u pythonu js game bez GUIa jer je tu bolje ai radit i da iman sve za posli analizu
- proba san fps based na oba ali je bolje timebased da bude fair game